In [ ]:
%run loadAndExtractPieces.ipynb

In [ ]:
# import the necessary packages
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import Input

from itertools import permutations

In [ ]:
# defines a list of permutations for integers 1-9 and returns a random permutation from the list
def rand_perm():
    p = list(permutations(range(1,10)))
    randP = np.random.randint(0, len(p))
    return (p[randP])

randPerm = rand_perm()

In [ ]:
#shape of one individual puzzle piece
inputShape = (66,66,3)

In [ ]:
def siamese_model(input_shape):
    
    inputs = Input(inputShape)

    p = layers.Conv2D(96, (11,11), activation="relu", strides=2,input_shape = inputShape)(inputs)
    p = layers.MaxPooling2D((3,3), strides = 2)(p)
    p = layers.Conv2D(256, (5,5), strides = 1, padding = "same", activation = "relu")(p)
    p = layers.MaxPooling2D((3,3), strides = 2)(p)
    p = layers.Conv2D(384, (3,3), strides = 1, padding = "same", activation = "relu")(p)
    p = layers.Conv2D(384, (3,3), strides = 1, padding = "same", activation = "relu")(p)
    p = layers.Conv2D(256, (3,3), strides = 1, padding = "same", activation = "relu")(p)
    p = layers.MaxPooling2D((3,3), strides = 2)(p)
    p = layers.Dropout(0.5)(p)
    p = layers.Dense(4096, activation = "relu")(p)
    
    model = Model(inputs, p)
    
    return model

In [ ]:
# all individual image pieces
p1 = Input(inputShape)
p2 = Input(inputShape)
p3 = Input(inputShape)
p4 = Input(inputShape)
p5 = Input(inputShape)
p6 = Input(inputShape)
p7 = Input(inputShape)
p8 = Input(inputShape)
p9 = Input(inputShape)

base = siamese_model(inputShape)

f1 = base(p1)
f2 = base(p2)
f3 = base(p3)
f4 = base(p4)
f5 = base(p5)
f6 = base(p6)
f7 = base(p7)
f8 = base(p8)
f9 = base(p9)


merged = layers.Concatenate()([f1, f2, f3, f4, f5, f6, f7, f8, f9])

merged = layers.Dense(4096, activation = 'relu')(merged)
merged = layers.Dense(81, activation = 'relu')(merged)
outputs = layers.Dense(9, activation = 'softmax')(merged)

model = Model(inputs=[p1, p2, p3, p4, p5, p6, p7, p8, p9], outputs=outputs)

#prints a summary of the model (layers, output shape, number of parameters)
base.summary()
model.summary()
    

In [ ]:
#used to visualise neural networks
from tensorflow.keras.utils import plot_model
plot_model(base, "siamese_model_base.png", show_shapes=True)

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
#the code below produces the following error:
#ValueError: Data cardinality is ambiguous:
#  x sizes: 66, 66, 66, 66, 66, 66, 66, 66, 66
#  y sizes: 384
#Make sure all arrays contain the same number of samples.

history = model.fit(
    (train_data[10][0],
    train_data[10][1],
    train_data[10][2],
    train_data[10][3],
    train_data[10][4],
    train_data[10][5],
    train_data[10][6],
    train_data[10][7],
    train_data[10][8]),
    train_labels,
    epochs=20,
    batch_size = 256)